In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import sys
import gc


In [3]:
input_folder = "C:/Users/toramoto/Desktop/PythonDesk/00_PythonStudy/Kag04/input/"
output_folder= "C:/Users/toramoto/Desktop/PythonDesk/00_PythonStudy/Kag04/output/"

building_metadata = pd.read_csv(input_folder + "building_metadata.csv")
test = pd.read_csv(input_folder + "test.csv")
weather_test = pd.read_csv(input_folder + "weather_test.csv")

### weatherの処理

In [4]:
# v3のfeature
# Weatherのmonth列を、中身をSeasonにする

winter = ['01','02','12']
spr_autm = ['03','04','10','11']
summer = ['05','06','07','08','09']

def seasonize(txt):
    if txt[5:7] in winter:
        return 1
    elif txt[5:7] in spr_autm:
        return 2
    elif txt[5:7] in summer:
        return 3
    else:
        return 0 # timestampはnullが無い筈なので、出番ないはず。これが出たら何かおかしい、ということ。


In [5]:
# time列を、3区分にまとめてみる
daytime = ["09","10","11","12","13","14","15","16","17","18"]
afternoon = ["19","20","21","22","23"]
midnight = ["00","01","02","03","04","05","06","07","08"]

def timezone(txt):
    if txt[11:13] in midnight:
        return 1
    elif txt[11:13] in afternoon:
        return 2
    elif txt[11:13] in daytime:
        return 3
    else:
        return 0

In [6]:
weather_test['month'] = weather_test['timestamp'].apply(seasonize)
weather_test['time'] = weather_test['timestamp'].apply(timezone)

In [7]:
# site_id,monthごとの平均値をDataframeに入れておく
waether_mean = weather_test.groupby(['site_id','month']).mean()
# site_idごとの平均値をDataframeに入れておく
waether_mean_s = weather_test.groupby(['site_id']).mean()

### buildingの処理

#### floor_countの欠損値を埋める

In [8]:
# 方針: floor countあたりの平均square_feetを出し、それでsquare_feetを割り算して欠損値を埋める
# 1.square_feet / floor_count　のカラムを作る 
building_metadata["sq_p_floor"]=building_metadata["square_feet"]/building_metadata["floor_count"]
# 2.square_feetをそれで割り、round upさせたものでfillna
building_metadata.floor_count = building_metadata.floor_count.fillna(round(building_metadata.square_feet/building_metadata.sq_p_floor.mean()+0.5))

#### year_builtの欠損値を埋める

In [9]:
# 方針
# primary_useの平均値で埋める
# それでもない場合はsite_idの平均値で埋める

# 平均値のdataframeを作っておく
building_metadata_mean_primary_use = building_metadata.groupby('primary_use').mean()
building_metadata_mean_site = building_metadata.groupby('site_id').mean()

In [10]:
# dataf = building_metadata[['primary_use','year_built']]
def filnaavg_year_p (dataf):
    p_use=dataf[0]
    column=dataf[1]
    if pd.isnull(column):
        return building_metadata_mean_primary_use.loc[p_use,"year_built"]
    else:
        return column

# dataf = building_metadata[['site_id','year_built']]
def filnaavg_year_s (dataf):
    site=dataf[0]
    column=dataf[1]
    if pd.isnull(column):
        return building_metadata_mean_site.loc[site,"year_built"]
    else:
        return column


In [11]:
# fill naする 

building_metadata.year_built = building_metadata[['primary_use','year_built']].apply(filnaavg_year_p, axis=1)
building_metadata.year_built = building_metadata[['site_id','year_built']].apply(filnaavg_year_s, axis=1)
building_metadata.year_built = building_metadata['year_built'].fillna(1970) #それでも埋まらない行があるので適当にダミー値入れる。

In [12]:
building_metadata.describe()

,site_id,building_id,square_feet,year_built,floor_count,sq_p_floor
count,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,355.000000
mean,6.952381,724.000000,92111.776398,1968.689217,4.559696,22778.805388
std,5.003432,418.434583,110769.950997,21.909009,4.619731,51395.993145
min,0.000000,0.000000,283.000000,1900.000000,1.000000,356.000000
25%,3.000000,362.000000,23012.000000,1962.579755,2.000000,6623.100000
50%,5.000000,724.000000,57673.000000,1970.904110,3.000000,12259.000000
75%,13.000000,1086.000000,115676.000000,1975.000000,6.000000,24190.402778
max,15.000000,1448.000000,875000.000000,2017.000000,38.000000,875000.000000


In [13]:
# 文字型をbool型にする
# joinしてからやるとメモリエラー起こしたので、joinする前にやる
building_metadata = pd.get_dummies(building_metadata, columns = ["primary_use"])

In [14]:
#### いらないカラムをドロップする
building_metadata.drop(['sq_p_floor'],axis=1,inplace=True)

### testデータとbuilding_metadataとweather_testをjoin

In [15]:
# result3=pd.merge(cpt_list_wk, result2df, on=['SOURCE_FC','DEST_FC','arc_branch_num'] ,how='inner')
test_pre1 = pd.merge(test, building_metadata, on='building_id' ,how='left')
#train_joined = pd.merge(train_pre1, weather_train, on=['site_id','timestamp'] ,how='left') #NaNがあるとモデル作成時にエラーになるため
test_joined = pd.merge(test_pre1, weather_test, on=['site_id','timestamp'] ,how='left')

In [16]:
del test
del test_pre1
del weather_test
del building_metadata
gc.collect()

63

欠損を埋める

In [17]:
test_joined['month'] = test_joined['timestamp'].apply(seasonize)
test_joined['time'] = test_joined['timestamp'].apply(timezone)

In [18]:
test_joined = pd.merge(test_joined, waether_mean, on=['site_id','month'] ,how='left')

In [19]:
test_joined["air_temperature_x"] = test_joined["air_temperature_x"].fillna(test_joined["air_temperature_y"])
test_joined["cloud_coverage_x"] = test_joined["cloud_coverage_x"].fillna(test_joined["cloud_coverage_y"])
test_joined["dew_temperature_x"] = test_joined["dew_temperature_x"].fillna(test_joined["dew_temperature_y"])
test_joined["precip_depth_1_hr_x"] = test_joined["precip_depth_1_hr_x"].fillna(test_joined["precip_depth_1_hr_y"])
test_joined["sea_level_pressure_x"] = test_joined["sea_level_pressure_x"].fillna(test_joined["sea_level_pressure_y"])
test_joined["wind_direction_x"] = test_joined["wind_direction_x"].fillna(test_joined["wind_direction_y"])
test_joined["wind_speed_x"] = test_joined["wind_speed_x"].fillna(test_joined["wind_speed_y"])

In [21]:
#test_joined.describe()

,row_id,building_id,meter,site_id,square_feet,year_built,floor_count,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,...,month,time_x,air_temperature_y,cloud_coverage_y,dew_temperature_y,precip_depth_1_hr_y,sea_level_pressure_y,wind_direction_y,wind_speed_y,time_y
count,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,...,4.169760e+07,4.169760e+07,4.169760e+07,4.071648e+07,4.169760e+07,3.840384e+07,4.013832e+07,4.169760e+07,4.169760e+07,4.169760e+07
mean,2.084880e+07,8.075824e+02,6.642857e-01,8.086134e+00,1.069469e+05,1.968743e+03,5.126471e+00,4.029412e-01,1.130252e-01,5.462185e-03,...,2.172603e+00,2.041667e+00,1.547144e+01,1.873092e+00,7.571125e+00,1.534530e+00,1.016219e+03,1.717824e+02,3.385399e+00,2.042495e+00
std,1.203706e+07,4.297680e+02,9.278067e-01,5.134712e+00,1.160888e+05,2.025187e+01,4.809433e+00,4.904891e-01,3.166236e-01,7.370447e-02,...,7.974721e-01,8.887804e-01,9.596607e+00,1.353304e+00,8.313756e+00,2.632006e+00,2.994474e+00,3.043020e+01,8.197809e-01,2.452739e-03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1.900000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,-6.382144e+00,4.928806e-03,-9.990652e+00,-1.027350e-02,1.007678e+03,1.068724e+02,1.924232e+00,2.038335e+00
25%,1.042440e+07,4.047500e+02,0.000000e+00,3.000000e+00,3.224350e+04,1.962580e+03,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e+00,1.000000e+00,8.188588e+00,7.201210e-01,4.219976e-01,5.537671e-01,1.014227e+03,1.488011e+02,2.796191e+00,2.041667e+00
50%,2.084880e+07,9.000000e+02,0.000000e+00,9.000000e+00,7.226250e+04,1.970904e+03,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e+00,2.000000e+00,1.796750e+01,1.798769e+00,7.908497e+00,8.415911e-01,1.016578e+03,1.730676e+02,3.384751e+00,2.041667e+00
75%,3.127320e+07,1.194250e+03,1.000000e+00,1.300000e+01,1.383875e+05,1.973282e+03,7.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,3.000000e+00,3.000000e+00,2.196639e+01,2.687164e+00,1.398087e+01,1.339076e+00,1.018090e+03,1.964869e+02,4.037014e+00,2.041950e+00
max,4.169760e+07,1.448000e+03,3.000000e+00,1.500000e+01,8.750000e+05,2.017000e+03,3.800000e+01,1.000000e+00,1.000000e+00,1.000000e+00,...,3.000000e+00,3.000000e+00,3.301844e+01,5.834382e+00,2.241273e+01,2.003377e+01,1.021118e+03,2.466473e+02,6.278532e+00,2.051296e+00


追加して使い終わった余計なカラムを削除

In [1]:
test_joined.drop(["air_temperature_y","cloud_coverage_y","dew_temperature_y","precip_depth_1_hr_y","sea_level_pressure_y","wind_direction_y","wind_speed_y","time_y"],axis=1)

NameError: name 'test_joined' is not defined

In [20]:
test_joined = pd.merge(test_joined, waether_mean_s, on=['site_id'] ,how='left')

In [21]:
test_joined.head().to_csv("tj.csv",index=None)

In [22]:
test_joined["cloud_coverage_x"] = test_joined["cloud_coverage_x"].fillna(test_joined["cloud_coverage"])
test_joined["precip_depth_1_hr_x"] = test_joined["precip_depth_1_hr_x"].fillna(test_joined["precip_depth_1_hr"])
test_joined["sea_level_pressure_x"] = test_joined["sea_level_pressure_x"].fillna(test_joined["sea_level_pressure"])


In [23]:
test_joined["cloud_coverage_x"] = test_joined["cloud_coverage_x"].fillna(test_joined["cloud_coverage_x"].mean())
test_joined["precip_depth_1_hr_x"] = test_joined["precip_depth_1_hr_x"].fillna(test_joined["precip_depth_1_hr_x"].mean())
test_joined["sea_level_pressure_x"] = test_joined["sea_level_pressure_x"].fillna(test_joined["sea_level_pressure_x"].mean())

92

In [27]:
test_joined_00 = test_joined.query('meter==0 & site_id==0')
test_joined_01 = test_joined.query('meter==0 & site_id==1')
test_joined_02 = test_joined.query('meter==0 & site_id==2')
test_joined_03 = test_joined.query('meter==0 & site_id==3')
test_joined_04 = test_joined.query('meter==0 & site_id==4')
test_joined_05 = test_joined.query('meter==0 & site_id==5')
test_joined_06 = test_joined.query('meter==0 & site_id==6')
test_joined_07 = test_joined.query('meter==0 & site_id==7')
test_joined_08 = test_joined.query('meter==0 & site_id==8')
test_joined_09 = test_joined.query('meter==0 & site_id==9')
test_joined_010 = test_joined.query('meter==0 & site_id==10')
test_joined_011 = test_joined.query('meter==0 & site_id==11')
test_joined_012 = test_joined.query('meter==0 & site_id==12')
test_joined_013 = test_joined.query('meter==0 & site_id==13')
test_joined_014 = test_joined.query('meter==0 & site_id==14')
test_joined_015 = test_joined.query('meter==0 & site_id==15')

test_joined_1X = test_joined.query('meter==1')
test_joined_2X = test_joined.query('meter==2')
test_joined_3X = test_joined.query('meter==3')

KeyError: MemoryError()

In [27]:
test_joined.describe()

KeyError: MemoryError()

In [25]:
test_joined["real_month"]=test_joined["timestamp"].str[5:7]

In [26]:
test_joined.to_csv("test_joined.csv",index=None)

KeyboardInterrupt: 

In [ ]:
train_joined.drop(['timestamp'],axis=1,inplace=True)